# Исследование надёжности заёмщиков

## План работ:
1.  Первый взгляд на данные.
2.  Предобработка данных.
3.  Обработка пропусков.
4.  Замена типа данных.
5.  Обработка дубликатов.
6.  Приведение к общему виду данных(лемматизация).
7.  Категоризация данных.
8.  Ответы на поставленные вопросы.
9.  Вывод.

### Описание данных:  
-  children — количество детей в семье
-  days_employed — общий трудовой стаж в днях
-  dob_years — возраст клиента в годах
-  education — уровень образования клиента
-  education_id — идентификатор уровня образования
-  family_status — семейное положение
-  family_status_id — идентификатор семейного положения
-  gender — пол клиента
-  income_type — тип занятости
-  debt — имел ли задолженность по возврату кредитов
-  total_income — ежемесячный доход
-  purpose — цель получения кредита

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
clients = pd.read_csv('datasets/data.csv')

In [2]:
clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


1.  Применил метод info:  
    1.1. Количество строк и столбцов.  
    1.2. В таблице данные трёх типов: float64(2), int64(5), object(5).  
    1.3. Есть незаполненные данные в столбцах days_employed и total_income (общий трудовой стаж в днях и ежемесячный доход).  
    1.4. Вероятно, исходя из одинакового колличества данных в столбцах days_employed и total_income, пропуски допущенны в идентичных строках.

In [3]:
display(clients)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


2.  С помощью  функции display:  
    2.1. Есть данные с отрицательым значением в стольбце days_employed.  
    2.2. В столбце education присутствует Разный регистр текста  
    2.3. В столбце purpose присутствуют по разному записанные одинаковые цели.

### Вывод

Данные не идеальны. Планируется замена пропущенных значений на средние значения в таблице. Причины пропусков - возможные ошибки базы данных при выгрузке или в заполнении анкеты заёмщиками. Возможно данные клиенты брали кредит по двум документам без трудовой книжки. Требуется приведение типа float  к int. Необходимо привести к единому регистру строковые значения, также привести их к единому виду.

### Шаг 2. Предобработка данных

### Обработка пропусков

In [4]:
miss = pd.isnull(clients)
omissions_true = miss[(miss['days_employed'] == True) & (miss['total_income'] == True)].count()
print(omissions_true)

children            2174
days_employed       2174
dob_years           2174
education           2174
education_id        2174
family_status       2174
family_status_id    2174
gender              2174
income_type         2174
debt                2174
total_income        2174
purpose             2174
dtype: int64


Проверил, что пропуски столбцов days_employed и total_income находятся попарно в строках.

In [5]:
print(clients['children'].count() - clients['days_employed'].count())

2174


Проверил количество пропусков

Предположительно, следствием является группы занятости заёмщиков:  
1.  Отсутсвие работы.  
2.  Предпринимательская деаятельность.  
3.  Стедент без опыта работы.

In [6]:
clients.groupby('income_type')['days_employed'].count()

income_type
безработный            2
в декрете              1
госслужащий         1312
компаньон           4577
пенсионер           3443
предприниматель        1
сотрудник          10014
студент                1
Name: days_employed, dtype: int64

Колличество заёмщиков с данными типами занятости слишком мало.
Рассмотрю таблицу исключительно с пропущенными данными:

In [7]:
clients = clients.fillna(0)
clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Заменил все пропуски в таблице методом fillna на 0.  
Проверил отсутсвтие пропусков в таблице после заполнения методом fillna

In [8]:
miss_table = clients[(clients['days_employed'] == 0) & (clients['total_income'] == 0)]
display(miss_table)


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,0.0,65,среднее,1,гражданский брак,1,M,пенсионер,0,0.0,сыграть свадьбу
26,0,0.0,41,среднее,1,женат / замужем,0,M,госслужащий,0,0.0,образование
29,0,0.0,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,0.0,строительство жилой недвижимости
41,0,0.0,50,среднее,1,женат / замужем,0,F,госслужащий,0,0.0,сделка с подержанным автомобилем
55,0,0.0,54,среднее,1,гражданский брак,1,F,пенсионер,1,0.0,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,0.0,47,Среднее,1,женат / замужем,0,M,компаньон,0,0.0,сделка с автомобилем
21495,1,0.0,50,среднее,1,гражданский брак,1,F,сотрудник,0,0.0,свадьба
21497,0,0.0,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,0.0,строительство недвижимости
21502,1,0.0,42,среднее,1,женат / замужем,0,F,сотрудник,0,0.0,строительство жилой недвижимости


Других закономерностей выявить не удаётся.

Проверю столбцы с числовыми значениями на уникальность:

In [9]:
clients['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

Изменю отрицательное колличество детей на положительное. Скорее всего аналог дефиса в заполненной анкете.

In [10]:
clients['children'] = clients['children'].apply(abs)
clients['children'].unique()

array([ 1,  0,  3,  2,  4, 20,  5])

Выявлено аномальное количество детей. Проверим количество клиентов с таким количеством детей:

In [11]:
clients['children'].value_counts()

0     14149
1      4865
2      2055
3       330
20       76
4        41
5         9
Name: children, dtype: int64

In [12]:
clients['children'].describe()

count    21525.000000
mean         0.543275
std          1.379876
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max         20.000000
Name: children, dtype: float64

Вероятность ошибки выше, чем наличие у 76 клиентов из 21510 по 20 детей. Скорее всего это остаточное явление записи 2.0. Необходимо заменить количество детей 20, на 2.  
Напишу для этого функцию:

In [13]:
def change_children(clients):
    num_children = clients['children']
    for row in clients:
        if num_children == 20:
            return 2
        else:
            return num_children
    

In [14]:
clients['children'] = clients.apply(change_children, axis=1)

In [15]:
clients['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

Данные по детям успешно изменены.

In [16]:
clients[['days_employed', 'dob_years']].sort_values('days_employed')

,days_employed,dob_years
16335,-18388.949901,61
4299,-17615.563266,61
7329,-16593.472817,60
17838,-16264.699501,59
16825,-16119.687737,64
...,...,...
7794,401663.850046,61
2156,401674.466633,60
7664,401675.093434,61
10006,401715.811749,69


Изменю все отрицательные значения столбца days_employed на положительные, т.к. отрицательное количество дней - скорее всего аналог дефиса.

In [17]:
clients['days_employed'] = clients['days_employed'].apply(abs)
clients[['days_employed']]

,days_employed
0,8437.673028
1,4024.803754
2,5623.422610
3,4124.747207
4,340266.072047
...,...
21520,4529.316663
21521,343937.404131
21522,2113.346888
21523,3112.481705


Отрицательные значения успешно заменены на положительные.

In [18]:
clients['days_employed'].describe()

count     21525.000000
mean      60156.419005
std      133355.929525
min           0.000000
25%         610.652074
50%        1808.053434
75%        4779.587738
max      401755.400475
Name: days_employed, dtype: float64

In [19]:
print(clients['days_employed'].max() / 365)

1100.6997273296713


Максимальный трудовой стаж d 1100 лет невозможен. Посмотрим на максимальный возраст клиента:

In [20]:
clients['dob_years'].describe() 

count    21525.000000
mean        43.293380
std         12.574584
min          0.000000
25%         33.000000
50%         42.000000
75%         53.000000
max         75.000000
Name: dob_years, dtype: float64

In [21]:
clients[clients['dob_years'] == 0]['dob_years'].count()

101

#### Обнаружена еще одна проблема - возраст, равный 0. В дальнейшем необходимо заполнить пропуски с учётом групп занятости..

In [22]:
print(f"Ожидаемый максимальный стаж лет: {clients['dob_years'].max() - 14}")
print(f"Ожидаемый максимальный стаж дней: {(clients['dob_years'].max() - 14) * 365}")

Ожидаемый максимальный стаж лет: 61
Ожидаемый максимальный стаж дней: 22265


Максимальный возраст клиента - 75 лет. Следовательно, стаж более 61 лет - ошибка данных, так как в России, трудовую книжку саводят минимум с 14 лет.  
Проверю количество аномально большого стажа, более 22265 дней, создам таблицу аномальных даненых:

In [23]:
anomal_days_employed = clients[clients['days_employed'] > 22265]
display(anomal_days_employed)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
18,0,400281.136913,53,среднее,1,вдовец / вдова,2,F,пенсионер,0,56823.777243,на покупку подержанного автомобиля
24,1,338551.952911,57,среднее,1,Не женат / не замужем,4,F,пенсионер,0,290547.235997,операции с коммерческой недвижимостью
25,0,363548.489348,67,среднее,1,женат / замужем,0,M,пенсионер,0,55112.757732,покупка недвижимости
30,1,335581.668515,62,среднее,1,женат / замужем,0,F,пенсионер,0,171456.067993,операции с коммерческой недвижимостью
...,...,...,...,...,...,...,...,...,...,...,...,...
21505,0,338904.866406,53,среднее,1,гражданский брак,1,M,пенсионер,0,75439.993167,сыграть свадьбу
21508,0,386497.714078,62,среднее,1,женат / замужем,0,M,пенсионер,0,72638.590915,недвижимость
21509,0,362161.054124,59,высшее,0,женат / замужем,0,M,пенсионер,0,73029.059379,операции с недвижимостью
21518,0,373995.710838,59,СРЕДНЕЕ,1,женат / замужем,0,F,пенсионер,0,153864.650328,сделка с автомобилем


Подозрительно, что в столбце income_type только пенсионеры.
Проверю минимальное значение среди аномальных:

In [24]:
anomal_days_employed['days_employed'].describe()

count      3445.000000
mean     365004.309916
std       21075.016396
min      328728.720605
25%      346639.413916
50%      365213.306266
75%      383246.444219
max      401755.400475
Name: days_employed, dtype: float64

Полученное минимальное значение значительно больше заданного.

Проверю на уникальность, какие группы занятости присутсвуют в аномальной таблице:

In [25]:
anomal_days_employed['income_type'].unique()

array(['пенсионер', 'безработный'], dtype=object)

Обнаружил, что аномальное значение стажа только у двух групп занятости:  
-  пенсионер
-  безработный  

Вероятно, такой аномальный трудовой стаж означает, что данные не вносились при заполнении анкеты либо вносились некорректно только для отсутвия пропусков. Вероятин и баг базы данных.  
Следует заменить аномальные значения на 0, чтобы потом заменить их на средние значения по группам занятости. Напишу для этого функцию и применю её:

In [26]:
def anomal_null(clients):
    days_employed = clients['days_employed']
    for row in clients:
        if days_employed > 22265:
            return 0
        else:
            return days_employed

In [27]:
clients['days_employed'] = clients.apply(anomal_null, axis=1)
clients[clients['days_employed'] > 22265]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


In [28]:
clients[clients['income_type'] == 'пенсионер']['days_employed'].unique()

array([0.])

In [29]:
 clients[clients['income_type'] == 'безработный']['days_employed'].unique()

array([0.])

После замены данных, в группах занятости, к сожалению не осталось ни одного значения. Придется заполнить их общими средними значениями.  
Аномальные значения в столбце days_employed убрал.  

#### Проанализирую нули в dob_years:

In [30]:
dob_years_zero = clients[clients['dob_years'] == 0]
clients[clients['dob_years'] == 0]['income_type'].unique()

array(['пенсионер', 'сотрудник', 'компаньон', 'госслужащий'], dtype=object)

Заполню нули в возрасте клиентов для каждой группы занятости отдельно:

In [31]:
mean_civil_servant_dob_years = clients[(clients['income_type'] == 'госслужащий') & (clients['dob_years'] != 0)]['dob_years'].mean()
mean_companion_dob_years = clients[(clients['income_type'] == 'компаньон') & (clients['dob_years'] != 0)]['dob_years'].mean()
mean_pensioner_dob_years = clients[(clients['income_type'] == 'пенсионер') & (clients['dob_years'] != 0)]['dob_years'].mean()
mean_employee_dob_years = clients[(clients['income_type'] == 'сотрудник') & (clients['dob_years'] != 0)]['dob_years'].mean()
print(f'Средний возраст госслужащих {mean_civil_servant_dob_years} лет:')
print(f'Средний возраст компаньонов {mean_companion_dob_years} лет:')
print(f'Средний возраст пенсионеров {mean_pensioner_dob_years} лет:')
print(f'Средний возраст сотрудников {mean_employee_dob_years} лет:')

Средний возраст госслужащих 40.80454232622161 лет:
Средний возраст компаньонов 39.854294175715694 лет:
Средний возраст пенсионеров 59.370959332638165 лет:
Средний возраст сотрудников 40.01898047722343 лет:


Напишу функцию для замены нулей на средний возраст:

In [32]:
def dob_years_no_zero(clients):
    yers = clients['dob_years']
    income_type = clients['income_type']
    for row in clients:
        if yers == 0:
            if income_type == 'госслужащий':
                return mean_civil_servant_dob_years
            elif income_type == 'компаньон':
                return mean_companion_dob_years
            elif income_type == 'пенсионер':
                return mean_pensioner_dob_years
            elif income_type == 'сотрудник':
                return mean_employee_dob_years
        else:
            return income_type

Применю функцию и проверю отсутсвие нулей в столбце dob_years:

In [33]:
clients['dob_years'] = clients.apply(dob_years_no_zero, axis = 1)
clients[clients['dob_years'] == 0]['dob_years'].unique()

array([], dtype=object)

Избавился от нулей в столбце dob_years.  
Продолжаю анализировать данные:

In [34]:
clients['education_id'].unique() 

array([0, 1, 2, 3, 4])

In [35]:
clients['family_status_id'].unique() 

array([0, 1, 2, 3, 4])

In [36]:
clients['debt'].unique() 

array([0, 1])

In [37]:
clients['total_income'].value_counts() 

0.000000         2174
133912.272223       1
182036.676828       1
122421.963500       1
198271.837248       1
                 ... 
133299.194693       1
115080.782380       1
84896.781597        1
153838.839212       1
220857.484722       1
Name: total_income, Length: 19352, dtype: int64

Заполню пропуски в базе средним значением каждого столбца в таблице для минимизации влияния пропусков на выводы. Поочередно найдем средние значения столбцов days_employed и total_income в зависимости от групп занятости, не забывая, что группы пенсионер и безработный в столбце days_employed, заполняются общим средним после заполнения всех остальных нулей:

In [38]:
clients.groupby('income_type')['days_employed', 'total_income'].count()

,days_employed,total_income
income_type,,
безработный,2,2
в декрете,1,1
госслужащий,1459,1459
компаньон,5085,5085
пенсионер,3856,3856
предприниматель,2,2
сотрудник,11119,11119
студент,1,1


In [39]:
mean_unemployed_ti = clients[(clients['income_type'] == 'безработный') & (clients['total_income'] != 0)]['total_income'].mean()
mean_on_maternity_de = clients[(clients['income_type'] == 'в декрете') & (clients['days_employed'] != 0)]['days_employed'].mean()
mean_on_maternity_ti = clients[(clients['income_type'] == 'в декрете') & (clients['total_income'] != 0)]['total_income'].mean()
mean_civil_servant_de = clients[(clients['income_type'] == 'госслужащий') & (clients['days_employed'] != 0)]['days_employed'].mean()
mean_civil_servant_ti = clients[(clients['income_type'] == 'госслужащий') & (clients['total_income'] != 0)]['total_income'].mean()
mean_companion_de = clients[(clients['income_type'] == 'компаньон') & (clients['days_employed'] != 0)]['days_employed'].mean()
mean_companion_ti = clients[(clients['income_type'] == 'компаньон') & (clients['total_income'] != 0)]['total_income'].mean()
mean_pensioner_ti = clients[(clients['income_type'] == 'пенсионер') & (clients['total_income'] != 0)]['total_income'].mean()
mean_businessman_de = clients[(clients['income_type'] == 'предприниматель') & (clients['days_employed'] != 0)]['days_employed'].mean()
mean_businessman_ti = clients[(clients['income_type'] == 'предприниматель') & (clients['total_income'] != 0)]['total_income'].mean()
mean_employee_de = clients[(clients['income_type'] == 'сотрудник') & (clients['days_employed'] != 0)]['days_employed'].mean()
mean_employee_ti = clients[(clients['income_type'] == 'сотрудник') & (clients['total_income'] != 0)]['total_income'].mean()
mean_student_de = clients[(clients['income_type'] == 'студент') & (clients['days_employed'] != 0)]['days_employed'].mean()
mean_student_ti = clients[(clients['income_type'] == 'студент') & (clients['total_income'] != 0)]['total_income'].mean()
print('Среднее total_income "безработный": {} р.'.format(mean_unemployed_ti))
print('Среднее days_employed "в декрете": {} д.'.format(mean_on_maternity_de))
print('Среднее total_income "в декрете": {} р.'.format(mean_on_maternity_ti))
print('Среднее days_employed "госслужащий": {} д.'.format(mean_civil_servant_de))
print('Среднее total_income "госслужащий": {} р.'.format(mean_civil_servant_ti))
print('Среднее days_employed "компаньон" {} д.'.format(mean_companion_de))
print('Среднее total_income "компаньон": {} р.'.format(mean_companion_ti))
print('Среднее total_income "пенсионер": {} р.'.format(mean_pensioner_ti))
print('Среднее days_employed "предприниматель": {} д.'.format(mean_businessman_de))
print('Среднее total_income "предприниматель": {} р.'.format(mean_businessman_ti))
print('Среднее days_employed "сотрудник": {} д.'.format(mean_employee_de))
print('Среднее total_income "сотрудник": {} р.'.format(mean_employee_ti))
print('Среднее days_employed "студент": {} д.'.format(mean_student_de))
print('Среднее total_income "студент": {} р.'.format(mean_student_ti))


Среднее total_income "безработный": 131339.7516762103 р.
Среднее days_employed "в декрете": 3296.7599620220594 д.
Среднее total_income "в декрете": 53829.13072905995 р.
Среднее days_employed "госслужащий": 3399.896901695741 д.
Среднее total_income "госслужащий": 170898.30992266268 р.
Среднее days_employed "компаньон" 2111.5243982977327 д.
Среднее total_income "компаньон": 202417.4614617771 р.
Среднее total_income "пенсионер": 137127.4656901654 р.
Среднее days_employed "предприниматель": 520.8480834953765 д.
Среднее total_income "предприниматель": 499163.1449470857 р.
Среднее days_employed "сотрудник": 2326.4992159717935 д.
Среднее total_income "сотрудник": 161380.2604878855 р.
Среднее days_employed "студент": 578.7515535382181 д.
Среднее total_income "студент": 98201.62531401133 р.


Напишу функции для заполнения пропусков в столбце подходящими значниями(кроме пенсионеров и безработных):

In [40]:
def fillna_mean_days_employed(clients):
    de = clients['days_employed']
    income_type = clients['income_type']
    for row in clients:
        if de == 0:
            if income_type == 'в декрете':
                return mean_on_maternity_de
            elif income_type == 'госслужащий':
                return mean_civil_servant_de
            elif income_type == 'компаньон':
                return mean_companion_de
            elif income_type == 'предприниматель':
                return mean_businessman_de
            elif income_type == 'сотрудник':
                return mean_employee_de
            elif income_type == 'студент':
                return mean_student_de
            else:
                return de
        else:
            return de
    

In [41]:
clients['days_employed'] = clients.apply(fillna_mean_days_employed, axis=1)

Написал и применил функцию для заполнения столбца days_employed (кроме пенсионеров и безработных).  
Аналогично сделаю для столбца total_income:

In [42]:
def fillna_mean_total_income(clients):
    ti = clients['total_income']
    income_type = clients['income_type']
    for row in clients:
        if ti == 0:
            if income_type == 'безработный':
                return mean_unemployed_ti
            elif income_type == 'в декрете':
                return mean_on_maternity_ti
            elif income_type == 'госслужащий':
                return mean_civil_servant_ti
            elif income_type == 'компаньон':
                return mean_companion_ti
            elif income_type == 'пенсионер':
                return mean_pensioner_ti
            elif income_type == 'предприниматель':
                return mean_businessman_ti
            elif income_type == 'сотрудник':
                return mean_employee_ti
            elif income_type == 'студент':
                return mean_student_ti
            else:
                return ti
        else:
            return ti

In [43]:
clients['total_income'] = clients.apply(fillna_mean_total_income, axis=1)

Применил функцию на все группы занятости(кроме безработных и пенсионеров в столбце days_employed)
Теперь выполню аналогичные действия с оставшимися нулями:

In [44]:
mean_unemployed_de = clients['days_employed'].mean()
mean_pensioner_de = clients['days_employed'].mean()

In [45]:
def fillna_mean_days_employed_pensioner_unemployed(clients):
    de = clients['days_employed']
    income_type = clients['income_type']
    for row in clients:
        if de == 0:
            if income_type == 'безработный':
                return mean_unemployed_de
            elif income_type == 'пенсионер':
                return mean_pensioner_de
        else:
            return de

In [46]:
clients['days_employed'] = clients.apply(fillna_mean_days_employed_pensioner_unemployed, axis=1)

После замены значений в столбцах, проверю нулевые значения:

In [47]:
clients[clients['total_income'] == 0]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


In [48]:
clients[clients['days_employed'] == 0]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


Пропуски и нули успешно заменены на средние значения по группам занятости.

### Вывод

Следует обратить внимание на малое количество некоторых групп занятости в таблице и пропуски. Возможны ошибки базы данных при выгрузке или в заполнении анкеты заёмщиками. Возможно данные клиенты брали кредит по двум документам без трудовой книжки. Запросить новый файл таблицы данных.

### Замена типа данных

Заменю тип данных float на int: методом astype(), т.к. метод to_numeric переводит данные во float, если есть хотябы одно число float.

In [49]:
clients['days_employed'] = clients['days_employed'].astype('int')
clients['total_income'] = clients['total_income'].astype('int')
display(clients)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,сотрудник,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,сотрудник,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,сотрудник,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,сотрудник,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,1931,пенсионер,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529,компаньон,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем
21521,0,1931,пенсионер,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем
21522,1,2113,сотрудник,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость
21523,3,3112,сотрудник,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля


### Вывод

Все данные в нужном формате, можно продолжать.

### Обработка дубликатов

Проверим наличие дубликатов методом duplicated, если есть посмотрим на их количество методом sum:

In [50]:
clients.duplicated().sum()

664

Дубликатов 664. Необходимо проверить все столбцы с типом данных object методом unique и при необходимости все регистры перевести в маленький регистр методом str.lower.  

Проверим столбец education:

In [51]:
clients['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

Присутвую одинаковые данные в разном регистре. Приведем всё к маленькому регистру и снова проверим дубликаты:

In [52]:
clients['education'] = clients['education'].str.lower()
clients.duplicated().sum()

762

In [53]:
clients['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [54]:
clients['family_status'] = clients['family_status'].str.lower()
clients.duplicated().sum()

762

In [55]:
clients['gender'].unique() 

array(['F', 'M', 'XNA'], dtype=object)

In [56]:
clients['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [57]:
clients['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Проблема с регистром обнаружена только в одном столбце - education. После перевода его данных к единому регистру, количество дубликоатов увеличилось с 664 до 762.

In [58]:
duplicates = clients.duplicated().sum() / clients['children'].count()
print('Процент дубликатов: {:.2%}'.format(duplicates))

Процент дубликатов: 3.54%


Уберу дубликаты методом drop_duplicates.

In [59]:
clients = clients.drop_duplicates().reset_index(drop=True)
clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20763 entries, 0 to 20762
Data columns (total 12 columns):
children            20763 non-null int64
days_employed       20763 non-null int64
dob_years           20763 non-null object
education           20763 non-null object
education_id        20763 non-null int64
family_status       20763 non-null object
family_status_id    20763 non-null int64
gender              20763 non-null object
income_type         20763 non-null object
debt                20763 non-null int64
total_income        20763 non-null int64
purpose             20763 non-null object
dtypes: int64(6), object(6)
memory usage: 1.9+ MB


### Вывод

Предположительно дубликаты появились из за повторной подачи анкеты, ошибки операционистов или сбоя базы данных.

### Лемматизация

В столбце purpose обнаружены слова, записанные в разных формах. Необходимо провести лемматизацию.
Выделяю все леммы в столбце purpose:

In [60]:
list_purpose = clients['purpose'].unique()
print(list_purpose)

['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильное образование'
 'высшее образование' 'покупка жилья для сдачи' 'на покупку автомобиля'
 'ремонт жилью' 'заняться высшим образованием']


In [61]:
from pymystem3 import Mystem
m = Mystem()
str_purpose = (', ').join(list_purpose)
lemmas = m.lemmatize(str_purpose)

str_purpose_done = ('').join(lemmas)
print(str_purpose_done)

покупка жилье, приобретение автомобиль, дополнительный образование, сыграть свадьба, операция с жилье, образование, на проведение свадьба, покупка жилье для семья, покупка недвижимость, покупка коммерческий недвижимость, покупка жилой недвижимость, строительство собственный недвижимость, недвижимость, строительство недвижимость, на покупка подержать автомобиль, на покупка свой автомобиль, операция с коммерческий недвижимость, строительство жилой недвижимость, жилье, операция со свой недвижимость, автомобиль, заниматься образование, сделка с подержать автомобиль, получение образование, автомобиль, свадьба, получение дополнительный образование, покупка свой жилье, операция с недвижимость, получение высокий образование, свой автомобиль, сделка с автомобиль, профильный образование, высокий образование, покупка жилье для сдача, на покупка автомобиль, ремонт жилье, заниматься высокий образование



Из полученной строки с целями кредита, выделю основные целей кредита:  
1.  недвижимость, жилье, жилой.  
2.  автомобиль.  
3.  образование.  
4.  свадьба.  
5.  коммерческий.  
6.  ремонт.  

Функция для распределения целей кредита:

In [62]:
def lemmatization_purpose(list):
    for perpose in list:
        if 'автомобиль' in perpose:
            print(perpose + ': автокредит')
        elif 'образование' in perpose:
            print(perpose + ': образование')
        elif 'свадьба' in perpose:
            print(perpose + ': свадьба')
        elif 'ремонт' in perpose:
            print(perpose + ': ремонт жилья')
        elif 'коммерческий' in perpose:
            print(perpose + ': коммерческая недвижимость')
        elif 'недвижимость' in perpose or 'жилье' in perpose or 'жилой' in perpose:
            print(perpose + ': жилая недвижимость')
list_purpose_done = str_purpose_done.split(', ')
lemmatization_purpose(list_purpose_done)

покупка жилье: жилая недвижимость
приобретение автомобиль: автокредит
дополнительный образование: образование
сыграть свадьба: свадьба
операция с жилье: жилая недвижимость
образование: образование
на проведение свадьба: свадьба
покупка жилье для семья: жилая недвижимость
покупка недвижимость: жилая недвижимость
покупка коммерческий недвижимость: коммерческая недвижимость
покупка жилой недвижимость: жилая недвижимость
строительство собственный недвижимость: жилая недвижимость
недвижимость: жилая недвижимость
строительство недвижимость: жилая недвижимость
на покупка подержать автомобиль: автокредит
на покупка свой автомобиль: автокредит
операция с коммерческий недвижимость: коммерческая недвижимость
строительство жилой недвижимость: жилая недвижимость
жилье: жилая недвижимость
операция со свой недвижимость: жилая недвижимость
автомобиль: автокредит
заниматься образование: образование
сделка с подержать автомобиль: автокредит
получение образование: образование
автомобиль: автокредит
свадь

Функция работает на лемматизированном списке целей.  
Перепишу функцию для применения на таблице данных:

In [63]:
def lemmatization_purpose(data):
    purpose = data['purpose']
    lemmas = m.lemmatize(purpose)
    for perpose in lemmas:
        if 'автомобиль' in perpose:
            return 'автокредит'
        elif 'образование' in perpose:
            return 'образование'
        elif 'свадьба' in perpose:
            return 'свадьба'
        elif 'ремонт' in perpose:
            return 'ремонт жилья'
        elif 'коммерческий' in perpose:
            return 'коммерческая недвижимость'
        elif 'недвижимость' in perpose or 'жилье' in perpose or 'жилой' in perpose:
            return 'жилая недвижимость'

Добавлю новый столбец lemm_purpose в таблицу, применив метод apply. Первый аргумент метода - функция распределения целей кредита, фторой аргумент axis - нужен для того, чтобы в функцию поступали данные в строковой форме:

In [64]:
clients['lemm_purpose'] = clients.apply(lemmatization_purpose, axis=1)
display(clients)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemm_purpose
0,1,8437,сотрудник,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,жилая недвижимость
1,1,4024,сотрудник,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автокредит
2,0,5623,сотрудник,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,жилая недвижимость
3,3,4124,сотрудник,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,1931,пенсионер,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20758,1,4529,компаньон,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,жилая недвижимость
20759,0,1931,пенсионер,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,автокредит
20760,1,2113,сотрудник,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,жилая недвижимость
20761,3,3112,сотрудник,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,автокредит


Столбец lemm_purpose с унифицированными целями кредита добавлен в основную таблицу.

### Вывод

Данные в столбце lemm_purpose позволяют составлять на их основе сводные тыблицы и подходят для дальнейшего анализа.

### Категоризация данных

Необходимо сгруппировать данные для ответа на поставленные вопросы по столбцу задолжности по кредиту и по столбцу с данными, на который поставлен вопрос.

- Сгруппирую данные по наличию детей. Так как в таблице наличие просрочки определяется единицей в столбце debt, а отсутствие нулём, посчитаю сумму просрочек:

In [65]:
children_grouping = clients.groupby('children').agg({'debt': ['count', 'sum']})
children_grouping

debt      
          count   sum
children             
0         13557  1051
1          4737   444
2          2090   202
3           329    27
4            41     4
5             9     0

Из таблицы видно, что больше кредитов взяли клиенты без детей.
Посмотрим на таблицу с просрочками:

- Сгруппирую данные по семейному положению и посмотрю просрочки:

In [66]:
family_status_groping = clients.groupby('family_status').agg({'debt': ['count', 'sum']})
family_status_groping

debt     
                       count  sum
family_status                    
в разводе               1182   85
вдовец / вдова           941   63
гражданский брак        4007  381
женат / замужем        11872  925
не женат / не замужем   2761  274

Из таблицы видно, что люди в браке взяли большее количество кредитов.

- Сгруппирую данные по уровню дохода:

Для этого необходимо разбить доходы на группы:  
1.  низкий уровень.  
2.  средний уровень.  
3.  высокий уровен.  
Методом qcut разобью на квантили все доходы:

In [67]:
quantils = clients['total_income']

In [68]:
clients['total_income_level'] = pd.qcut(quantils, 3, labels=['низкий доход', 'средний доход', 'высокий доход'])
display(clients)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemm_purpose,total_income_level
0,1,8437,сотрудник,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,жилая недвижимость,высокий доход
1,1,4024,сотрудник,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автокредит,низкий доход
2,0,5623,сотрудник,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,жилая недвижимость,средний доход
3,3,4124,сотрудник,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,высокий доход
4,0,1931,пенсионер,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,средний доход
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20758,1,4529,компаньон,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,жилая недвижимость,высокий доход
20759,0,1931,пенсионер,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,автокредит,средний доход
20760,1,2113,сотрудник,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,жилая недвижимость,низкий доход
20761,3,3112,сотрудник,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,автокредит,высокий доход


In [69]:
total_level_groping = pd.pivot_table(clients, index = ['total_income_level'], values = 'debt', aggfunc = ['count', 'sum'])
total_level_groping

,count,sum
,debt,debt
total_income_level,,
низкий доход,6921,560
средний доход,6921,652
высокий доход,6921,516


Т.к. группировали по методу qcut, группы получились равными.

Сгруппируем общую таблицу по целям кредита:

In [70]:
purpose_grouping = clients.groupby('lemm_purpose').agg({'debt': ['count', 'sum']})
purpose_grouping

debt     
                          count  sum
lemm_purpose                        
автокредит                 4189  401
жилая недвижимость         8602  646
коммерческая недвижимость  1273   99
образование                3911  368
ремонт жилья                589   35
свадьба                    2199  179

Больше кредитов взяли на жилую недвижимость.

### Вывод

Созданы сводные таблицы по категориям данных для дальнейшей работы с ними.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Выясним процентное соотношение просроченных кредитов по количеству детей, добавив в таблицу chidren_table столбец с процентами просрочек:

In [71]:
children_grouping['percent'] = children_grouping['debt']['sum'] / children_grouping['debt']['count'] * 100
children_grouping

debt         percent
          count   sum          
children                       
0         13557  1051  7.752453
1          4737   444  9.373021
2          2090   202  9.665072
3           329    27  8.206687
4            41     4  9.756098
5             9     0  0.000000

По полученным данным видно, что процент просроченных кредитов больше у клиентов с детьми(кроме клиентов с 5 детьми).
Выясним процент просроченных кредитов у клиентов хотябы с одним ребенком:

In [72]:
summ_credit_with_children = clients[clients['children'] > 0]['debt'].count()
summ_expired_credit_with_children = clients[clients['children'] > 0]['debt'].sum()
percent_expired_with_children = summ_expired_credit_with_children / summ_credit_with_children
summ_credit_without_children = clients[clients['children'] == 0]['debt'].count()
summ_expired_credit_without_children = clients[clients['children'] == 0]['debt'].sum()
percent_expired_without_children = summ_expired_credit_without_children / summ_credit_without_children
print('Процент просроченных кредитов у клиентов с детьми: {:.1%}'.format(percent_expired_with_children))
print('Процент просроченных кредитов у клиентов без детей: {:.1%}'.format(percent_expired_without_children))

Процент просроченных кредитов у клиентов с детьми: 9.4%
Процент просроченных кредитов у клиентов без детей: 7.8%


### Вывод

Клиенты с детьми задержали выплату на 1.6%  чаще, чем клиенты без детей. Зависимость присутствует.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Добавим в таблицу family_status_table столбец с процентами просрочек:

In [73]:
family_status_groping['percent'] = family_status_groping['debt']['sum'] / family_status_groping['debt']['count'] * 100
family_status_groping

debt        percent
                       count  sum          
family_status                              
в разводе               1182   85  7.191201
вдовец / вдова           941   63  6.695005
гражданский брак        4007  381  9.508360
женат / замужем        11872  925  7.791442
не женат / не замужем   2761  274  9.923941

### Вывод

Клиента 'вдовец / вдова' имеют меньше всего просрочек по кредиту (6.7%).  
Клиенты 'в разводе' и 'женат / замужем' хуже контролируют кредиты(7.2% и 7.8%).   
Хуже всего дела обстаят у клиентов в статусе 'гражданский брак' и 'не женат / не замужем' (9.5% и 9.9%).  
Зависимеость присутсвует.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Добавим в таблицу total_income_level_table столбец с процентами просрочек:

In [74]:
total_level_groping['percent'] = total_level_groping['sum', 'debt'] / total_level_groping['count', 'debt'] * 100
total_level_groping

,count,sum,percent
,debt,debt,
total_income_level,,,
низкий доход,6921,560,8.091316
средний доход,6921,652,9.420604
высокий доход,6921,516,7.455570


In [75]:
percent_expired_hight = clients[clients['total_income_level'] == 'высокий доход']['debt'].sum() / clients[clients['total_income_level'] == 'высокий доход']['debt'].count() 
percent_expired_midle = clients[clients['total_income_level'] == 'средний доход']['debt'].sum() / clients[clients['total_income_level'] == 'средний доход']['debt'].count() 
percent_expired_low = clients[clients['total_income_level'] == 'низкий доход']['debt'].sum() / clients[clients['total_income_level'] == 'низкий доход']['debt'].count() 
print("Процент просрочек у клинтов с высоким доходом: {:.1%}".format(percent_expired_hight))
print("Процент просрочек у клинтов со средним доходом: {:.1%}".format(percent_expired_midle))
print("Процент просрочек у клинтов с низким доходом: {:.1%}".format(percent_expired_low))

Процент просрочек у клинтов с высоким доходом: 7.5%
Процент просрочек у клинтов со средним доходом: 9.4%
Процент просрочек у клинтов с низким доходом: 8.1%


### Вывод

Клиенты с высоким доходом лучше контролируют выплаты по кредиту.  
Клиенты со средним уровнем дохода часто имеют просрочки.  
Зависимость от дохода присутсвует.

- Как разные цели кредита влияют на его возврат в срок?

Добавим в таблицу purpose_table столбец с процентами просрочек:

In [76]:
purpose_grouping['percent'] =  purpose_grouping['debt']['sum'] / purpose_grouping['debt']['count'] * 100
purpose_grouping

debt        percent
                          count  sum          
lemm_purpose                                  
автокредит                 4189  401  9.572690
жилая недвижимость         8602  646  7.509881
коммерческая недвижимость  1273   99  7.776905
образование                3911  368  9.409358
ремонт жилья                589   35  5.942275
свадьба                    2199  179  8.140064

In [77]:
students = clients[clients['income_type'] == 'студент']['lemm_purpose']
students.count()

1

In [78]:
education = clients[clients['lemm_purpose'] == 'образование']['lemm_purpose']
education.count()

3911

In [79]:
education = clients[clients['lemm_purpose'] == 'образование']['income_type']
education.value_counts('income_type')

сотрудник      0.515981
компаньон      0.238814
пенсионер      0.179749
госслужащий    0.065456
Name: income_type, dtype: float64

Из-за  того, что некоторых групп студентов слишком мало, не удается проанализировать цель образование. Считаю логичным, что на образование, подавляющей группой клиентов должны быть студенты(или их родители, что противоречет моей теории=)).

### Вывод

Наименьшее количество просрочек получилось с целью "ремонт жилья"(5.9%).  
Наибольшее - "автокредит"(9.6%). Возможно потому, что автомобили быстро устаревают либо приходят в негодность после аварий и клиенты не хотят(не могут) выплачивать данный вид кредита. 'Образование' должно быть в группе риска скорее всего из-за бедных студентов, но в таблице студентов влишком мало.
Цель кредита влияет на выплату.

### Шаг 4. Общий вывод

Проанализировал данные банка. Присутвуют ошибки базы - слишком большие и отрицательные значения в столбце трудового стажа, подозрительноь мало некоторых групп занятости. Нужно запросить данные заново. Избавился от пропусков данных. Максимально сохранил данные для анализа. Унифицировал цели кредита. Разбил группу клиентов на уровни дохода по квантилям. Подготовил сводные таблицы для отчета. Раньше считал наиболее надежными клиентов с детьми  и в браке. Из проделанной работы могу сделать вывод:  
Наиболее рискованная группа клиентов:  
-  с детьми
-  не женат / не замужем
-  со средним уровнем дохода  
-  с целью - автомобиль или образование.  

Наименее рискованная группа клиентов:  
-  без детей
-  овдовевшие 
-  с высоким уровнем дохода  
-  с целью - ремонт жилья.